# Fact or Fake: News Analysis

### Data Mining 334
### Alex Laughlin, Xandre Clementsmith

---
## Imports

In [170]:
# required for running on jupyterlabs (Xandre)
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Xirailuyo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Xirailuyo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [226]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import sklearn
import math
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import re
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem.snowball import SnowballStemmer

## Data Transforms

In [227]:
#pull in data

#just the politifact data for now
df_p_real = pd.read_csv('politifact_real.csv')
df_p_real['true/false'] = True
df_p_real.head(2)

,id,news_url,title,tweet_ids,true/false
0,politifact14984,http://www.nfib-sbet.org/,National Federation of Independent Business,967132259869487105\t967164368768196609\t967215...,True
1,politifact12944,http://www.cq.com/doc/newsmakertranscripts-494...,comments in Fayetteville NC,942953459\t8980098198\t16253717352\t1668513250...,True


In [228]:
df_p_fake = pd.read_csv('politifact_fake.csv')
df_p_fake['true/false'] = False
df_p_fake.head(2)

,id,news_url,title,tweet_ids,true/false
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...,False
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...,False


In [229]:
#Join tables together
frames = [df_p_real, df_p_fake]
df = pd.concat(frames)
df = df.reset_index(drop=True)

## Regularization

In [230]:
stemmer = SnowballStemmer("english")

tokenizer = nltk.RegexpTokenizer(r"\w+")

#remove special characters
df['title'] = df.apply(lambda row: re.sub('[^A-Za-z0-9 ]+', '', str(row['title'])), axis =1)

#tokenize words from title
df['tokenized_sents'] = df.apply(lambda row: tokenizer.tokenize(row['title']), axis=1)

#remove stop words from tokenized titles
df['tokens_without_stopwords'] = df['tokenized_sents'].apply(lambda x: [item for item in x if item not in stop_words])

#stem tokenized words without stopwords
df['tokens_stemmed']=df['tokens_without_stopwords'].apply(lambda x : [stemmer.stem(y) for y in x])

#lemmatize tokenized words without stopwords
lemmatizer = nltk.stem.WordNetLemmatizer()
df['tokens_lemmatized'] = df['tokens_without_stopwords'].apply(lambda x : [lemmatizer.lemmatize(y) for y in x])
df.head(5)

,id,news_url,title,tweet_ids,true/false,tokenized_sents,tokens_without_stopwords,tokens_stemmed,tokens_lemmatized
0,politifact14984,http://www.nfib-sbet.org/,National Federation of Independent Business,967132259869487105\t967164368768196609\t967215...,True,"[National, Federation, of, Independent, Business]","[National, Federation, Independent, Business]","[nation, feder, independ, busi]","[National, Federation, Independent, Business]"
1,politifact12944,http://www.cq.com/doc/newsmakertranscripts-494...,comments in Fayetteville NC,942953459\t8980098198\t16253717352\t1668513250...,True,"[comments, in, Fayetteville, NC]","[comments, Fayetteville, NC]","[comment, fayettevill, nc]","[comment, Fayetteville, NC]"
2,politifact333,https://web.archive.org/web/20080204072132/htt...,Romney makes pitch hoping to close deal Elect...,NaN,True,"[Romney, makes, pitch, hoping, to, close, deal...","[Romney, makes, pitch, hoping, close, deal, El...","[romney, make, pitch, hope, close, deal, elect...","[Romney, make, pitch, hoping, close, deal, Ele..."
3,politifact4358,https://web.archive.org/web/20110811143753/htt...,Democratic Leaders Say House Democrats Are Uni...,NaN,True,"[Democratic, Leaders, Say, House, Democrats, A...","[Democratic, Leaders, Say, House, Democrats, A...","[democrat, leader, say, hous, democrat, are, u...","[Democratic, Leaders, Say, House, Democrats, A..."
4,politifact779,https://web.archive.org/web/20070820164107/htt...,Budget of the United States Government FY 2008,89804710374154240\t91270460595109888\t96039619...,True,"[Budget, of, the, United, States, Government, ...","[Budget, United, States, Government, FY, 2008]","[budget, unit, state, govern, fy, 2008]","[Budget, United, States, Government, FY, 2008]"


## Feature Extraction

### Term Frequency - Inverse Document Frequency

In [231]:
# frequency matrix approx. = # of documents with word (word rarely appears twice in title)
def frequency_matrix(column):
    freq_matrix = {}

    for row in range(column.shape[0]):
        words = row
        for word in column.iloc[row,0]:
            if word in freq_matrix:
                freq_matrix[word] += 1
            else:
                freq_matrix[word] = 1

    return freq_matrix

In [232]:
# set regularized column variable to df-column of choice 
colvar = 'tokens_lemmatized'
reg_col = df[[colvar]]

# frequency matrix df
fm = pd.DataFrame.from_dict(frequency_matrix(reg_col), orient='index')
idf = fm.copy()

# inverse document frequency df
for row in range(df.shape[0]):
    idf.iloc[row] = math.log(df.shape[0] / idf.iloc[row])
    
idf.head(5)

,0
National,4.323186
Federation,6.962243
Independent,6.962243
Business,5.575949
comment,5.863631


In [233]:
# BUG HERE
# Shouldn't change colvar (column variable) column, only idf, due to deep copy. However, it is changing colvar
df['idf'] = reg_col.copy(deep=True)
for row in range(df[['idf']].shape[0]):
    for item in range(len(df.loc[row,'idf'])):
        token = df.loc[row,'idf'][item]
        df.loc[row,'idf'][item] = idf.loc[token][0]

C:\Users\Xirailuyo\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Xirailuyo\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Machine Learning

In [235]:
# PLACEHOLDER needs to be replaced with a predictor dataframe
X_train, X_test, y_train, y_test = train_test_split(
    df.loc[:,"PLACEHOLDER"], df[[r'true/false']], test_size=0.33, random_state=42)

### Linear Regression

In [236]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(X_train, y_train)
lr.score(X_train, y_train)
lr.coef_
lr.intercept_

ValueError: Expected 2D array, got 1D array instead:
array=[ 2.77371881  5.5759491   3.44067064  3.58715782  4.24263529  3.56742777
  8.63805887  5.22191491  2.125       6.11704687  2.10033006  3.81712799
  2.33366529  3.2199697   9.75307281  1.79927823  2.52531637         nan
  5.67536985  6.86168382  3.24875962  5.63361411  2.75811776  3.52573001
  3.16973135  2.61443357  2.74794392  4.47200233  2.46701375  2.08062117
  3.3587946   5.83453861  4.58987942  4.23340766  5.2854096   4.04604191
  3.00642857  3.81552522  5.72804232  5.77000808  3.08969876  3.36994602
  4.09522227  1.59475617  6.26909628  5.31592612  3.08462745  1.
  3.72438037  3.05630526  3.61377562  4.58987942  2.56917238  5.06927795
  5.29100394  2.78426852  5.50016923  1.          2.63784848  3.
  3.46018933  5.95590554  3.63683011  4.93331856  1.75        2.59735482
  3.80065671  1.875       5.44513707  6.95260383  2.92522797  3.63260977
  5.10567933  5.5759491   3.67440214  2.22581164  4.2795729   5.96366604
  2.5419